In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import seaborn as sns
from scipy.optimize import minimize
from pprint import pprint
from matplotlib.patches import Wedge
import time
import pickle
import os
import importlib.util

In [ ]:

base_path = '/content/drive/MyDrive/SYNC_Projects/IS'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define relative paths from the base path
pathPARAMS = os.path.join(base_path, 'data/')
pathFIGURES = os.path.join(base_path, 'figures/')
pathCODE = os.path.join(base_path, 'code/')
pathSIM = os.path.join(base_path, 'runs_eps/')

os.makedirs(pathSIM, exist_ok=True)  # Create the directory if it doesn't exist


# Verify paths and imported data
print("pathDATA:", pathPARAMS)
print("pathCODE:", pathCODE)
print("pathFIGURES:", pathFIGURES)
print("pathSIM:", pathSIM)


module_path = os.path.join(pathCODE, "MonodGillespieIS_multispecies.py")
spec = importlib.util.spec_from_file_location("mg", module_path)
mg = importlib.util.module_from_spec(spec)
print(dir(mg))


spec.loader.exec_module(mg)

print("Module loaded successfully!")

Mounted at /content/drive
pathDATA: /content/drive/MyDrive/SYNC_Projects/IS/data/
pathCODE: /content/drive/MyDrive/SYNC_Projects/IS/code/
pathFIGURES: /content/drive/MyDrive/SYNC_Projects/IS/figures/
pathSIM: /content/drive/MyDrive/SYNC_Projects/IS/runs_eps/
['__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']
Module loaded successfully!


## Evolutionary model


### Parameters


In [ ]:

# Define parameters for each mutation-transposition level
num_mutationsSNP = 3  # Number of mutation levels
num_mutationsIS = 3  # Number of transposition levels

# Set initial resource, max time, and antibiotic concentration
initial_resource = 1.0
simulation_time = 24.0
B0=1e6

# Shared parameters
num_days = 45
dilution_factor = 0.1  # dilution factor applied each day

num_reps = 50
days=range(0, num_days + 1)

strainIDs = ["K253", "K168", "K037", "K241", "K209"]  # Species IDs
species_colors=['orange','blue',  'purple', 'green', 'yellow']

### Load params

In [ ]:

mg.num_species = len(strainIDs)  # Number of species
strains_species = mg.load_strain_parameters(strainIDs, pathPARAMS)
num_species = len(strainIDs)  # Number of species (e.g., "E" and "K")

# Example: Print strains_species to confirm
print("Strains for all species:")
for species_idx, strains in strains_species.items():
    print(f"Species {species_idx}:")
    print(f"  Plasmid-free strains: {strains['0']}")
    print(f"  Plasmid-bearing strains: {strains['p']}")


Plasmid-free parameters for strain K253 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K253_0.pkl

Plasmid-bearing parameters for strain K253 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K253_p.pkl

Plasmid-free parameters for strain K168 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K168_0.pkl

Plasmid-bearing parameters for strain K168 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K168_p.pkl

Plasmid-free parameters for strain K037 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K037_0.pkl

Plasmid-bearing parameters for strain K037 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K037_p.pkl

Plasmid-free parameters for strain K241 loaded successfully from:
/content/drive/MyDrive/SYNC_Projects/IS/data/params_K241_0.pkl

Plasmid-bearing parameters for strain K241 loaded successfully from:
/content/dri

### Define initial conditions

In [ ]:

initial_values_by_strain = {
    "K253": {"0": 1e6, "p": 0},     # Initial densities
    "K168": {"0": 1e6, "p": 0},     # Initial densities
    "K037": {"0": 1e6, "p": 0},      # Initial densities
     "K241": {"0": 1e6, "p": 100},      # Initial densities
     "K209": {"0": 1e6, "p": 0},      # Initial densities
}

# Define initial conditions
populations_0, populations_p = mg.initialize_populations(
    num_species=num_species,
    num_mutationsSNP=num_mutationsSNP,
    num_mutationsIS=num_mutationsIS,
    strainIDs=strainIDs,
    initial_values_by_strain=initial_values_by_strain
)


### Run simulation

In [ ]:

antibiotic_concentration=0.2
eps_values = np.logspace(-11, -7, num=21)

expe_results = []  # Will store list of dicts per eps value


for eps in eps_values:
    print("Conjugation_rate=",eps)


    eps_str = f"{eps:.2e}"  # Format eps nicely
    filename = f"expe_result_eps_{eps_str}.pkl"
    filepath = os.path.join(pathSIM, filename)

    if os.path.exists(filepath):
        print(f"Loading existing simulation for eps = {eps_str}")
        with open(filepath, 'rb') as f:
            expe_results_dict = pickle.load(f)

    else:

      # Define the plasmid transmission matrix
      plasmid_matrix = np.array([
          [eps, eps, eps, eps, eps],  # K253
          [eps, eps, eps, eps, eps],  # K168
          [eps, eps, eps, eps, eps],   # K037
          [eps, eps, eps, eps, eps],   # K241
          [eps, eps, eps, eps, eps],   # K209
      ])



      #Run simulation
      results = mg.runManySimulations(
          plasmid_matrix=plasmid_matrix,
          num_reps=num_reps,
          runSimulationIS_multi_species=mg.runSimulationIS_multi_species,
          strains=strains_species,
          initial_populations_0=populations_0,
          initial_populations_p=populations_p,
          num_days=num_days,
          antibiotic_concentration=antibiotic_concentration,
          initial_resource=initial_resource,
          simulation_time=simulation_time,
          dilution_factor=dilution_factor
      )

      #Compute and store statistics
      diversity_metrics = mg.computeStrainDiversity(results)
      stats = mg.computeMutationAndISTranspositionStatistics(results, num_mutationsSNP, num_mutationsIS, num_species)
      cumulative_densities_free, cumulative_densities_bearing = mg.computeCumulativeStrainDensities(results, num_species, num_days)
      plasmid_fraction_stats = mg.computePlasmidFraction(results)
      expe_results_dict = {
          "eps": eps,
          "strain_densities": {},
          "plasmid_fraction": plasmid_fraction_stats.get("Plasmid Fraction", np.nan),
          "plasmid_free_density": plasmid_fraction_stats.get("Plasmid-Free Density", np.nan),
          "plasmid_bearing_density": plasmid_fraction_stats.get("Plasmid-Bearing Density", np.nan),
          "IS_Cumulative_Frequency": stats.get("IS_Cumulative_Frequency", np.nan),
          "SNP_Cumulative_Frequency": stats.get("SNP_Cumulative_Frequency", np.nan),
      }
      #Save final densities of each strain
      for species_idx in range(num_species):
              free_density = cumulative_densities_free[species_idx, -1]
              bearing_density = cumulative_densities_bearing[species_idx, -1]
              strain_label = strainIDs[species_idx]
              expe_results_dict["strain_densities"][strain_label] = {
                  "free": free_density,
                  "bearing": bearing_density
      }

      with open(filepath, 'wb') as f:
          pickle.dump(expe_results_dict, f)

    expe_results.append(expe_results_dict)

    # Run print functions
    #mg.printCumulativeStrainDensities(cumulative_densities_free, cumulative_densities_bearing, num_species, strainIDs)
    #mg.printPlasmidFraction(plasmid_fraction_stats)
    #mg.printStrainDiversity(diversity_metrics)
    #mg.printMutationAndISTranspositionStatistics(stats)

    # Plot results
    #expe_path='' #If empty, dont save figures
    #mg.plotResults(plasmid_matrix, populations_0, populations_p, expe_results_dict, num_mutationsSNP, num_mutationsIS, num_days, num_species, species_colors, strainIDs, pathFIGURES, expe_path, antibiotic_concentration)


Conjugation_rate= 1e-11
..................................................Conjugation_rate= 1.5848931924611107e-11
..................................................Conjugation_rate= 2.5118864315095823e-11
..................................................Conjugation_rate= 3.9810717055349695e-11
..................................................Conjugation_rate= 6.309573444801942e-11
..................................................Conjugation_rate= 1e-10
..................................................Conjugation_rate= 1.584893192461111e-10
..................................................Conjugation_rate= 2.511886431509582e-10
..................................................Conjugation_rate= 3.9810717055349694e-10
..................................................Conjugation_rate= 6.309573444801942e-10
..................................................Conjugation_rate= 1e-09
..................................................Conjugation_rate= 1.584893192461111e-09
..............

In [ ]:
#eps=1e-7
#plasmid_matrix = np.array([
#        [eps, eps, eps, eps, eps],  # K253
#        [eps, eps, eps, eps, eps],  # K168
#        [eps, eps, eps, eps, eps],   # K037
#        [eps, eps, eps, eps, eps],   # K241
#        [eps, eps, eps, eps, eps],   # K209
#    ])
#mg.plot_HGT_network(plasmid_matrix, strainIDs, species_colors, outPath='', node_alpha=0.45)

In [ ]:
def plot_eps_vs_plasmid_fraction(expe_results, outPath=''):
    """
    Plots conjugation rate (eps) vs plasmid fraction (%).

    Parameters:
    expe_results (list of dicts): Output from simulations, each dict must contain 'eps' and 'plasmid_fraction'.
    outPath (str): Path to save the figure (optional).

    Returns:
    None
    """
    eps_values = [entry["eps"] for entry in expe_results]

    # Ensure we extract a single value (mean) if it's a list or array
    plasmid_fractions = []
    for entry in expe_results:
        val = entry["plasmid_fraction"]
        if isinstance(val, (list, np.ndarray)):
            plasmid_fractions.append(np.mean(val))
        else:
            plasmid_fractions.append(val)

    fig, ax = plt.subplots(figsize=(5.5, 4))

    ax.plot(eps_values, 100 * np.array(plasmid_fractions), lw=2, linestyle='-', color='black')
    ax.set_xscale("log")
    ax.set_ylim([-5, 105])
    ax.set_xlabel("Conjugation Rate (ε)", fontsize=16)
    ax.set_ylabel("Plasmid Fraction (%)", fontsize=16)
    ax.grid(False)
    ax.tick_params(axis='both', labelsize=14)
    plt.tight_layout()

    if outPath:
        filename = f"{outPath}eps_vs_fraction.pdf"
        plt.savefig(filename)
        print(f"Saved plot to {filename}")
        plt.show()
    else:
        plt.show()

plot_eps_vs_plasmid_fraction(expe_results[:-2], outPath=pathFIGURES)


In [ ]:

def plot_eps_vs_frequencies(expe_results, outPath=''):
    """
    Plots conjugation rate (eps) vs plasmid fraction, IS frequency, and SNP frequency.

    Parameters:
    expe_results (list of dicts): Output from simulations, each dict must contain 'eps',
                                  'plasmid_fraction', 'IS_Cumulative_Frequency', and 'SNP_Cumulative_Frequency'.
    outPath (str): Path to save the figure (optional).

    Returns:
    None
    """
    eps_values = [entry["eps"] for entry in expe_results]
    IS_frequencies = [entry["IS_Cumulative_Frequency"] for entry in expe_results]
    SNP_frequencies = [entry["SNP_Cumulative_Frequency"] for entry in expe_results]

    fig, ax = plt.subplots(figsize=(6, 4))

    ax.plot(eps_values, IS_frequencies,  linestyle='-', lw=2, label='IS Frequency', color='#810F7C')
    ax.plot(eps_values, SNP_frequencies, linestyle='-', lw=2,  label='SNP Frequency', color='#69acecff')

    ax.set_xscale("log")
    ax.set_xlabel("Conjugation Rate (ε)", fontsize=16)
    ax.set_ylabel("Frequency", fontsize=16)

    ax.tick_params(axis='both', labelsize=14)
    ax.grid(False)
    ax.legend(fontsize=12)
    plt.tight_layout()

    if outPath:
        filename = f"{outPath}eps_vs_frequencies.pdf"
        plt.savefig(filename)
        print(f"Saved plot to {filename}")
        plt.show()
    else:
        plt.show()

plot_eps_vs_frequencies(expe_results[:-2], outPath=pathFIGURES)

In [ ]:
def plot_strain_densities_vs_eps(expe_results, strainIDs, species_colors, outPath=''):
    """
    Plots plasmid-free (dotted) and plasmid-bearing (solid) densities for each strain vs. conjugation rate (eps).

    Parameters:
    expe_results (list of dicts): Simulation outputs, each dict must include 'eps' and 'strain_densities'.
    strainIDs (list of str): List of strain names (e.g., ["K253", "K168", ...]).
    species_colors (list of str): List of color names for each strain.
    outPath (str): Path to save the figure (optional).
    """
    eps_values = [entry["eps"] for entry in expe_results]

    # Initialize storage per strain
    free_densities = {strain: [] for strain in strainIDs}
    bearing_densities = {strain: [] for strain in strainIDs}

    for entry in expe_results:
        for strain in strainIDs:
            free = entry["strain_densities"][strain]["free"]
            bearing = entry["strain_densities"][strain]["bearing"]
            free_densities[strain].append(free)
            bearing_densities[strain].append(bearing)

    fig, ax = plt.subplots(figsize=(7, 4.5))

    for idx, strain in enumerate(strainIDs):
        color = species_colors[idx]
        # Solid for plasmid-bearing
        ax.plot(eps_values, bearing_densities[strain], color=color, lw=2, label=f"{strain} (bearing)")
        # Dotted for plasmid-free
        ax.plot(eps_values, free_densities[strain], color=color, lw=2, linestyle='dotted', label=f"{strain} (free)")

    ax.set_xscale("log")
    ax.set_xlabel("Conjugation Rate (ε)", fontsize=16)
    ax.set_ylabel("Final Density", fontsize=16)
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(False)
    #ax.grid(True, which='both', linestyle='--', alpha=0.5)
    #ax.legend(fontsize=10, bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0)

    handles = [plt.Rectangle((0, 0), 1, 1, color=species_colors[i], edgecolor='black') for i in range(num_species)]
    ax.legend(handles, strainIDs, fontsize=12, loc='upper left', bbox_to_anchor=(1.05, 1), title="Strains")

    plt.tight_layout()

    if outPath:
        filename = f"{outPath}eps_vs_strains.pdf"
        plt.savefig(filename, bbox_inches='tight')
        print(f"Saved plot to {filename}")
        plt.show()
    else:
        plt.show()

plot_strain_densities_vs_eps(expe_results[:-2], strainIDs, species_colors, outPath=pathFIGURES)
